# General Code Setup

## -Preperation-

this section takes data and prepares it for CPR

### --BaseExtra--

this module includes basic items used in preperation

In [1]:
import random
import numpy as np
import pandas as pd

In [14]:
def randomvector(size, maximum):
    return [random.randrange(1, maximum, 1) for i in range(size)]

In [29]:
type(randomvector(5, 4))

list

In [18]:
def testcheck(x, vset):
    if type(pd.DataFrame()) == pd.core.frame.DataFrame
        for v in vset.values:
            if np.equal(x, v).all().all():
                return True
    return False

In [19]:
testcheck(3, test)

AttributeError: 'list' object has no attribute 'values'

In [ ]:
def closestvector(target, A):
    newv = [np.linalg.norm(np.array(np.array(target)-np.array(a))) for a in A]
    return np.argmin(newv)

In [2]:
def farthestvector(target, A):
    newv = [np.linalg.norm(np.array(np.array(target)-np.array(a))) for a in A]
    return np.argmax(newv)

In [ ]:
def move(x, a, v):
    n = abs(x-a)*v
    if a>x:
        return x+n
    elif a<x:
        return x-n
    else:
        return x

In [ ]:
def multimove(X, A, v):
    newvec = list()
    for n, x in enumerate(X):
        newvec.append(move(x, A[n], v))
    return newvec

### --LaptopPrep--

this module is made for the laptop data example

### --Prep--

this module is used on data to prepare it for cpr using 

## -Excecution-

this section takes the data and trains the cpr model

### --Decision--

this module takes decisions and uses it to train a prediction module

### --SimBot--

this module simulates decisions based on a predetermined expected result

### --DeepRun--

this module goes through all possible results with simbot

## -Diagnostics-

this section takes the cpr model and runs diagnosis on it

### --Graph--

this module makes a 

### --Diagnose--

this checks the results of a deep run